In [1]:
%load_ext autoreload
%autoreload 2

In [30]:
import polars as pl

In [21]:
from lobgpt.hdb import get_dataset, reconstruct_snapshots

# <font color="grey">Data loading and handling</font>

In [3]:
dl = get_dataset("mic")

In [4]:
df = dl.load_book("RTSX", ["200305.000000", "200305.120000"])

In [5]:
df.head()

tst,tsr,delivery_delay_ms,event_code,is_buy,prc,prc_diff,vol,order_id,order_id_offset
i64,i64,i64,i32,bool,i32,i32,i32,i64,i64
1583394591163000000,1583394591280000000,117,2,true,133010,0,1,25731155910498,1
1583394591763000000,1583394591877000000,114,2,true,132320,-690,10,25731157862755,1952258
1583394599623000000,1583394599735000000,112,2,false,139240,6920,1,25731153882619,-2027878
1583394612131000000,1583394612240000000,109,2,false,136220,-3020,1,25731149088954,-6821543
1583394616176000000,1583394616292000000,116,2,false,137420,1200,1,25731153478528,-2431969


In [54]:
dl = get_dataset("lobster")

In [40]:
df = dl.load_book("AAPL", ["120621.093000", "120621.103000"])

In [41]:
df.head()

tst,event_code,is_buy,prc,vol,direction,order_id
i64,i8,bool,f64,i32,i8,i64
1340271000004241176,1,true,585.33,18,1,16113575
1340271000004260640,1,true,585.32,18,1,16113584
1340271000004447484,1,true,585.31,18,1,16113594
1340271000025551909,1,false,585.91,18,-1,16120456
1340271000025579546,1,false,585.92,18,-1,16120480


In [42]:
df.shape

(91997, 7)

In [55]:
df = dl.load_book("AAPL", ["120621.093000", "120621.103000"], type="snapshot")

In [56]:
df.head()

tst,asks[0].price,asks[0].amount,bids[0].price,bids[0].amount,asks[1].price,asks[1].amount,bids[1].price,bids[1].amount,asks[2].price,asks[2].amount,bids[2].price,bids[2].amount,asks[3].price,asks[3].amount,bids[3].price,bids[3].amount,asks[4].price,asks[4].amount,bids[4].price,bids[4].amount,asks[5].price,asks[5].amount,bids[5].price,bids[5].amount,asks[6].price,asks[6].amount,bids[6].price,bids[6].amount,asks[7].price,asks[7].amount,bids[7].price,bids[7].amount,asks[8].price,asks[8].amount,bids[8].price,bids[8].amount,…,bids[40].amount,asks[41].price,asks[41].amount,bids[41].price,bids[41].amount,asks[42].price,asks[42].amount,bids[42].price,bids[42].amount,asks[43].price,asks[43].amount,bids[43].price,bids[43].amount,asks[44].price,asks[44].amount,bids[44].price,bids[44].amount,asks[45].price,asks[45].amount,bids[45].price,bids[45].amount,asks[46].price,asks[46].amount,bids[46].price,bids[46].amount,asks[47].price,asks[47].amount,bids[47].price,bids[47].amount,asks[48].price,asks[48].amount,bids[48].price,bids[48].amount,asks[49].price,asks[49].amount,bids[49].price,bids[49].amount
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1340271000004241176,585.94,200.0,585.33,18.0,585.98,200.0,585.3,150.0,586.1,200.0,585.1,5.0,586.89,300.0,585.01,89.0,586.95,50.0,584.97,5.0,587.0,100.0,584.93,300.0,587.1,10.0,584.65,300.0,587.39,100.0,584.53,300.0,587.65,1160.0,584.38,200.0,…,400.0,null,0.0,500.0,100.0,null,0.0,483.03,100.0,null,0.0,481.13,20.0,null,0.0,480.0,102.0,null,0.0,478.59,100.0,null,0.0,473.65,100.0,null,0.0,468.96,200.0,null,0.0,468.95,1900.0,null,0.0,468.94,400.0
1340271000004260640,585.94,200.0,585.33,18.0,585.98,200.0,585.32,18.0,586.1,200.0,585.3,150.0,586.89,300.0,585.1,5.0,586.95,50.0,585.01,89.0,587.0,100.0,584.97,5.0,587.1,10.0,584.93,300.0,587.39,100.0,584.65,300.0,587.65,1160.0,584.53,300.0,…,20.0,null,0.0,520.0,400.0,null,0.0,500.0,100.0,null,0.0,483.03,100.0,null,0.0,481.13,20.0,null,0.0,480.0,102.0,null,0.0,478.59,100.0,null,0.0,473.65,100.0,null,0.0,468.96,200.0,null,0.0,468.95,1900.0
1340271000004447484,585.94,200.0,585.33,18.0,585.98,200.0,585.32,18.0,586.1,200.0,585.31,18.0,586.89,300.0,585.3,150.0,586.95,50.0,585.1,5.0,587.0,100.0,585.01,89.0,587.1,10.0,584.97,5.0,587.39,100.0,584.93,300.0,587.65,1160.0,584.65,300.0,…,1.0,null,0.0,523.8,20.0,null,0.0,520.0,400.0,null,0.0,500.0,100.0,null,0.0,483.03,100.0,null,0.0,481.13,20.0,null,0.0,480.0,102.0,null,0.0,478.59,100.0,null,0.0,473.65,100.0,null,0.0,468.96,200.0
1340271000025551909,585.91,18.0,585.33,18.0,585.94,200.0,585.32,18.0,585.98,200.0,585.31,18.0,586.1,200.0,585.3,150.0,586.89,300.0,585.1,5.0,586.95,50.0,585.01,89.0,587.0,100.0,584.97,5.0,587.1,10.0,584.93,300.0,587.39,100.0,584.65,300.0,…,1.0,null,0.0,523.8,20.0,null,0.0,520.0,400.0,null,0.0,500.0,100.0,null,0.0,483.03,100.0,null,0.0,481.13,20.0,null,0.0,480.0,102.0,null,0.0,478.59,100.0,null,0.0,473.65,100.0,null,0.0,468.96,200.0
1340271000025579546,585.91,18.0,585.33,18.0,585.92,18.0,585.32,18.0,585.94,200.0,585.31,18.0,585.98,200.0,585.3,150.0,586.1,200.0,585.1,5.0,586.89,300.0,585.01,89.0,586.95,50.0,584.97,5.0,587.0,100.0,584.93,300.0,587.1,10.0,584.65,300.0,…,1.0,2000.0,100.0,523.8,20.0,null,0.0,520.0,400.0,null,0.0,500.0,100.0,null,0.0,483.03,100.0,null,0.0,481.13,20.0,null,0.0,480.0,102.0,null,0.0,478.59,100.0,null,0.0,473.65,100.0,null,0.0,468.96,200.0


In [57]:
messages = dl.load_book("AAPL", ["120621.093000", "120621.103000"])
ob = dl.load_book("AAPL", ["120621.093000", "120621.103000"], type="snapshot")

In [58]:
df = reconstruct_snapshots(messages, ob.head(1))

In [59]:
df.shape

(91997, 201)

In [60]:
difdf = df-ob

In [61]:
numeric_cols = [c for c, dtype in difdf.schema.items() if dtype.is_numeric()]

non_zero_rows = difdf.filter(
    pl.any_horizontal([pl.col(c) != 0 for c in numeric_cols])
)

In [62]:
non_zero_rows

tst,asks[0].price,asks[0].amount,bids[0].price,bids[0].amount,asks[1].price,asks[1].amount,bids[1].price,bids[1].amount,asks[2].price,asks[2].amount,bids[2].price,bids[2].amount,asks[3].price,asks[3].amount,bids[3].price,bids[3].amount,asks[4].price,asks[4].amount,bids[4].price,bids[4].amount,asks[5].price,asks[5].amount,bids[5].price,bids[5].amount,asks[6].price,asks[6].amount,bids[6].price,bids[6].amount,asks[7].price,asks[7].amount,bids[7].price,bids[7].amount,asks[8].price,asks[8].amount,bids[8].price,bids[8].amount,…,bids[40].amount,asks[41].price,asks[41].amount,bids[41].price,bids[41].amount,asks[42].price,asks[42].amount,bids[42].price,bids[42].amount,asks[43].price,asks[43].amount,bids[43].price,bids[43].amount,asks[44].price,asks[44].amount,bids[44].price,bids[44].amount,asks[45].price,asks[45].amount,bids[45].price,bids[45].amount,asks[46].price,asks[46].amount,bids[46].price,bids[46].amount,asks[47].price,asks[47].amount,bids[47].price,bids[47].amount,asks[48].price,asks[48].amount,bids[48].price,bids[48].amount,asks[49].price,asks[49].amount,bids[49].price,bids[49].amount
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.24,0.0,0.0,0.0
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.24,0.0,0.0,0.0
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.24,0.0,0.0,0.0
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.24,0.0,0.0,0.0
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.24,0.0,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,900.0,-0.13,-45.0,0.06,-1000.0,-0.13,950.0,0.01,1090.0,-0.1,-623.0,0.05,-190.0,-0.08,30.0,0.05,132.0,-0.08,518.0,0.06,88.0,-0.02,900.0,0.1,78.0,-0.02,200.0,0.12,92.0,0.0,100.0,0.13,-840.0,0.0,1098.0,0.12,-2489.0
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,900.0,-0.13,-45.0,0.06,-1000.0,-0.13,950.0,0.01,1090.0,-0.1,-623.0,0.05,-190.0,-0.08,30.0,0.05,132.0,-0.08,518.0,0.06,88.0,-0.02,900.0,0.1,78.0,-0.02,200.0,0.12,92.0,0.0,100.0,0.13,-840.0,0.0,1098.0,0.12,-2489.0
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,900.0,-0.02,2995.0,0.06,-1000.0,-0.13,-45.0,0.01,1090.0,-0.13,950.0,0.05,-190.0,-0.1,-623.0,0.05,